In [ ]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEXMEGA'
CRYPTO_TARGET = 'AVRCRYPTOLIB'

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET"
cd ../../hardware/victims/firmware/simpleserial-aes
make PLATFORM=$1 CRYPTO_TARGET=$2

In [ ]:
%run "../Setup_Scripts/Setup_Generic.ipynb"

In [ ]:
scope.scopetype.cwFirmwareConfig[0xACE2].loader.setFPGAMode("debug")
print("Mode: " + str(scope.scopetype.cwFirmwareConfig[0xACE2].loader._release_mode))
print("Mode: " + str(scope.scopetype.cwFirmwareConfig[0xACE2].loader.fpga_bitstream()))

In [ ]:
%run "../Setup_Scripts/Setup_Generic.ipynb"

In [ ]:
fw_path = '../../hardware/victims/firmware/simpleserial-aes/simpleserial-aes-{}.hex'.format(PLATFORM)
cw.program_target(scope, prog, fw_path)

In [ ]:
import holoviews as hv
from holoviews.streams import Pipe, Buffer
import pandas as pd
from tqdm import tnrange
import numpy
from chipwhisperer.common.traces import Trace
import chipwhisperer.analyzer as cwa
import chipwhisperer as cw
from scipy.ndimage.filters import uniform_filter1d


class real_time_plot:
    def __init__(self, plot_len, title="Plot"):
        hv.extension('bokeh')
        st = pd.DataFrame({'y':[]}, columns=['y'])
        self.dfstream = Buffer(st, length=plot_len, index=False)
        curve_dmap = hv.DynamicMap(hv.Curve, streams=[self.dfstream])
        curve_dmap.opts(tools=["hover"], width=800, height=600, title=title)
#         curve_dmap.relabel('Tick formatters').opts(xformatter='$%.8f', yformatter='$%.2f', width=500) 
        display(curve_dmap)
        self.curve = curve_dmap
    def send(self, data):
        if hasattr(data, 'wave'):
            d = data.wave
        else:
            d = data
        self.dfstream.send(pd.DataFrame({'y':d}))
        
def powercycle_target(scope):
    scope.io.tio1 = "gpio_low"
    time.sleep(0.0001)
    scope.io.tio1 = "gpio_high"

def setup_ref_collection():
    scope.clock.adc_src = "clkgen_x1"

    scope.adc.basic_mode = "rising_edge"
    scope.trigger.module = "basic"
    scope.trigger.triggers = "tio4"
    print("trigger pin: %s" % scope.trigger.triggers)

    scope.adc.offset = offset
    scope.adc.samples = nr_samples
    scope.adc.presamples=0
    scope.gain.gain = 50
    
def collect_reference_trace(traces, nr_samples, offset):
    setup_ref_collection()

    new_trace = numpy.zeros(traces*nr_samples)
    for i in tnrange(traces, desc='Capturing traces'):
        cur_offset = int(offset + nr_samples*i)
        scope.adc.offset = cur_offset

        scope.arm()
        powercycle_target(scope)
        scope.capture()

        trace = scope.get_last_trace()
        if trace is None:
            continue
        new_trace = numpy.append(new_trace, trace)
    plot.send(new_trace)
    return new_trace

def collect_offline_traces(traces, offset):
    setup_ref_collection()
    scope.adc.offset = offset
    
    project = cw.create_project("projects/sad_traces", overwrite = True)

    for i in tnrange(traces, desc='Capturing traces'):
        scope.arm()
        powercycle_target(scope)
        scope.capture()

        wave = scope.get_last_trace()
        if trace is None:
            continue
        project.traces.append(Trace(wave, None, None, None))
#         plot.send(wave)
    project.save()
    return project

class edge_count:
    def __init__(self, edge_type, settling_time, trig_threshold, edge_num):
        self.edge_type = edge_type
        self.settling_time = settling_time
        self.trig_threshold = trig_threshold
        self.edge_num = edge_num
        
    def run(self, trace):
#         kernel = numpy.ones(length)/length
        
        # https://stackoverflow.com/questions/13728392/moving-average-or-running-mean
        ret = uniform_filter1d(trace, size=self.settling_time)
        
#         print(f"trace len: {length}")
#         print(f"kernel len: {len(kernel)}")
#         print(f"kernel: {kernel}")
        print(f"trace: {trace}")
        print(f"ret: {ret}")
        
        return ret

offset = int(24400*0)
traces = 100
nr_samples = 24400

In [ ]:
# plot = real_time_plot(plot_len=scope.adc.samples*traces)
# trace = collect_reference_trace(traces, nr_samples, offset)
project = collect_offline_traces(traces, 400000-2000)

In [ ]:
project = cw.open_project("projects/sad_traces")

In [ ]:
import chipwhisperer.analyzer.preprocessing as cwa

# pre_allign_plot = real_time_plot(plot_len=nr_samples)

# ref_wave = numpy.zeros(nr_samples)
# for i, el in enumerate(ref_wave):
#     if i < 4000:
#         ref_wave[i] = 0.2
        
# ref_trace = Trace(ref_wave, None, None, None)

# new_p = cw.create_project("projects/tmp", overwrite=True)
# new_p.traces.append(ref_trace)
# for trace in project.traces:
#     new_p.traces.append(trace)

resync_sad = cwa.ResyncSAD(new_p)
ref_trace_idx = 0
resync_sad.max_shift = 3950
resync_sad.ref_trace = ref_trace_idx
resync_sad.target_window = (3950, 4050)
resync_sad._debugReturnSad = True

# pre_allign_plot.send(new_p.traces[ref_trace_idx])
edge_count = edge_count(None, 100, None, None)
print(resync_sad)
# project = new_p

In [ ]:
alligned_project = resync_sad.preprocess()

In [ ]:
import time 

original = real_time_plot(plot_len=nr_samples, title="original")
original.send(project.traces[0])
pre_shift = real_time_plot(plot_len=nr_samples, title="pre-shift")
post_shift = real_time_plot(plot_len=nr_samples, title="post-shift")
for i in range(0, 6):
    if i > 5:
        break
    pre_shift.send(project.traces[i])
    post_shift.dfstream.clear()
    convol = edge_count.run(project.traces[i].wave)
    
    post_shift.send(convol)
    
    
#     zeros_pre = numpy.count_nonzero(project.traces[i].wave == 0)
#     zeros_post = numpy.count_nonzero(trace.wave == 0)
#     print(f"zeros pre shift: {zeros_pre}")
#     print(f"indices 0 pre shift: {numpy.where(project.traces[i].wave == 0)}")
#     print(f"zeros post shift: {zeros_post}")
#     print(f"difference: {abs((zeros_pre - zeros_post))}")
#     print(f"indices 0 pre shift: {numpy.where(trace.wave == 0)}")
    input("next?")

In [ ]:
point = int(561180-64) #int(1.444E5)
# scope.adc.stream_mode = False
scope.SAD.reference = trace[point:point+128]
hv.Curve(scope.SAD.reference).opts(tools=["hover"], width=900, height=600)

In [ ]:
scope.SAD.threshold = 5000

In [ ]:
scope.SAD.start()
print(scope.SAD.check_status())
print(scope.SAD.getThreshold())
scope.trigger.module = "SAD"
scope.adc.basic_mode = "rising_edge"
# scope.trigger
scope.adc.samples = nr_samples
scope.adc.presamples = 2000

# print(scope)

In [ ]:
plot = real_time_plot(plot_len=20000)

from tqdm import tnrange
# ktp = cw.ktp.Basic()

# key, text = ktp.next()

for i in tnrange(10, desc='Capturing traces'):    
    powercycle_target(scope)
    scope.arm()
    scope.capture()
    trace = scope.get_last_trace()
    if trace is None:
        continue
    plot.send(trace)

In [ ]:
from tqdm import tnrange
project = cw.create_project("projects/Tutorial_B5", overwrite = True)
for i in tnrange(100, desc='Capturing traces'):
    key, text = ktp.next()  # manual creation of a key, text pair can be substituted here
    trace = cw.capture_trace(scope, target, text, key)
    if trace is None:
        continue
    project.traces.append(trace)
project.save()